In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

In [6]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [7]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [8]:
print("test linbe")

test linbe


In [9]:
X_train_tensor

tensor([[-1.2720,  2.1126, -1.2796,  ..., -1.0504, -0.9913, -0.6604],
        [-0.7374, -0.2176, -0.7242,  ...,  0.0752, -0.1616,  0.3891],
        [-1.8574,  1.4610, -1.8289,  ..., -1.0298, -0.1890,  1.2589],
        ...,
        [-0.1787,  1.2839, -0.2058,  ...,  0.2460,  0.9744, -0.2770],
        [-1.0408, -0.2199, -1.0680,  ..., -1.4573, -0.3446, -0.9248],
        [-0.7175, -0.4867, -0.7443,  ..., -0.1079, -0.5619, -0.7239]])

In [10]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [11]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [17]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
        nn.Sigmoid()
    )
    

  def forward(self, features):

    out = self.network(features)

    return out

In [18]:
learning_rate = 0.1
epochs = 25

In [19]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [22]:
num_epochs = 20

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()   # training mode
    epoch_train_loss = 0

    for batch_features, batch_labels in train_loader:
        optimizer.zero_grad()

        # forward
        y_pred = model(batch_features)
        loss = loss_function(y_pred, batch_labels.view(-1, 1).float())

        # backward
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()

    # average train loss
    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)



    # -------------------------
    # VALIDATION STEP
    # -------------------------
    model.eval()
    epoch_val_loss = 0

    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            y_pred = model(batch_features)
            loss = loss_function(y_pred, batch_labels.view(-1, 1).float())
            epoch_val_loss += loss.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}")


Epoch [1/20] | Train Loss: 0.0291 | Val Loss: 0.0911
Epoch [2/20] | Train Loss: 0.0274 | Val Loss: 0.1265
Epoch [3/20] | Train Loss: 0.0244 | Val Loss: 0.1040
Epoch [4/20] | Train Loss: 0.0214 | Val Loss: 0.0965
Epoch [5/20] | Train Loss: 0.0197 | Val Loss: 0.1121
Epoch [6/20] | Train Loss: 0.0156 | Val Loss: 0.2023
Epoch [7/20] | Train Loss: 0.0145 | Val Loss: 0.1387
Epoch [8/20] | Train Loss: 0.0137 | Val Loss: 0.1198
Epoch [9/20] | Train Loss: 0.0162 | Val Loss: 0.1180
Epoch [10/20] | Train Loss: 0.0084 | Val Loss: 0.1246
Epoch [11/20] | Train Loss: 0.0076 | Val Loss: 0.1273
Epoch [12/20] | Train Loss: 0.0079 | Val Loss: 0.1327
Epoch [13/20] | Train Loss: 0.0074 | Val Loss: 0.1548
Epoch [14/20] | Train Loss: 0.0061 | Val Loss: 0.1446
Epoch [15/20] | Train Loss: 0.0055 | Val Loss: 0.1294
Epoch [16/20] | Train Loss: 0.0049 | Val Loss: 0.1342
Epoch [17/20] | Train Loss: 0.0052 | Val Loss: 0.1469
Epoch [18/20] | Train Loss: 0.0040 | Val Loss: 0.1937
Epoch [19/20] | Train Loss: 0.0034 | 

In [23]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9844


## ***Create the validations dataset with traing data***

In [ ]:
from sklearn.model_selection import train_test_split

# 1. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], 
    df.iloc[:, 0], 
    test_size=0.2, 
    random_state=42
)

# 2. Train/Validation split (from training)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size=0.2, 
    random_state=42
)

# 3. Tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# 4. Datasets
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
val_dataset = CustomDataset(X_val_tensor, y_val_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

# 5. DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)
